# Probabilistic Model of Bilateral Lymphatic Spread in Head and Neck

Cancer

Roman Ludwig [](https://orcid.org/0000-0001-9434-328X) (University of Zurich, University Hospital Zurich)  
Yoel Perez Haas (University of Zurich, University Hospital Zurich)  
Jan Unkelbach [](https://orcid.org/0000-0002-4275-990X) (University of Zurich, University Hospital Zurich)

In current practice, large parts of the contralateral lymph system are included in the elective clinical target volume (CTV-N) of oropharyngeal squamous cell carcinoma (OPSCC) patients. Guidelines on how to define the CTV-N are mostly based on prevalences of nodal involvement in the different lymph node levels (LNLs) and do not consider the detailed, individual diagnosis of a patient.

In this work, we present a probabilistic model for bilateral lymphatic tumor progression in OPSCC. Its aim is to predict the personal risk for occult disease in any LNL, given the patient’s clincal diagnosis, T-category, and lateralization of the primary tumor. The model represents each of the LNLs I, II, III, IV, V, and VII of both sides of the neck as a hidden binary random variable that are connected to the tumor and among each other via arcs that correspond to spread probabilities per time-step. These spread probability rates are learned via Markov chain Monte Carlo (MCMC) sampling from a dataset of 833 OPSCC patients.

The model is able to describe the data well and may in the future be used to only include LNLs in a CTV-N that show a risk above a specific threshold for that particular patient’s diagnosis. E.g., it predicts the risk for occult disease in contralateral LNL II to be below 5% unless the tumor crosses the mid-sagittal line.

In [2]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
import upsetplot

from lymph import models
from lyscripts import utils
from lyscripts.plot.utils import COLORS

from scripts import shared, paths, COL, get_lnl_cols

simple_model = shared.get_model("simple", load_samples=True)

In [3]:
# width of figures. May depend on number of text columns
full = 17 # cm
half = full
# half =  8 # cm

# Introduction

When treating head and neck squamous cell carcinomas (HNSCC) with radiotherapy or surgery, not only the primary tumor and the clinically detected lymph node metastases are targeted. Per current guidelines, large volumes of the neck - lymph node levels (LNLs) - are part of the the elective clinical target volume (CTV-N) \[@gregoire_ct-based_2003;@gregoire_delineation_2014;@gregoire_delineation_2018;@eisbruch_intensity-modulated_2002;@biau_selection_2019;@chao_determination_2002;@vorwerk_guidelines_2011;@ferlito_elective_2009\]. This is to minimize the risk of regional recurrences due to missed microscopic disease which cannot be detected by current in-vivo imaging modalities such as computed tomography (CT), magnetic resonance imaging (MRI), or positron emission tomography (PET).

Minimizing this risk of missing occult disease in the lymph drainage region must be balanced against the toxicity related to unnecessary treatment of healthy tissue.

The aforementioned guidelines for the definition of the CTV-N quantify this risk via the overall prevalence of lymph node metastases in the LNLs. Based on this, they recommend extensive irradiation of both sides of the neck for a majority of patients. However, this prevalence the guidelines are based on does not take into account the individual state of tumor progression, which may have a great impact on a patient’s personal risk for occult disease in a given LNL. For example, a patient with substantial nodal involvement in the ipsilateral side and an advanced primary tumor that crosses the mid-sagittal plane would receive extensive elective irradiation of the contralateral LNLs II, III, and IVa. However, if the patient presented with no clinically detectable nodal disease (cN0) and only a small, clearly lateralized T1 tumor, the same contralateral CTV-N would be recommended \[@biau_selection_2019\].

To better quantify this personalized risk we have previously developed an intuitive, probabilistic hidden Markov model \[@ludwig_hidden_2021;@ludwig_modelling_2023\] which was originally based on a conceptually similar Bayesian network model \[@pouymayou_bayesian_2019\]. So far, these models were limited to the ipsilateral side of the neck. The main focus of this work will be to extend the formalism of these previous works to include risk predictions for the contralateral side as well. We believe this holds the potential to greatly reduce the electively irradiated volumes in the contralateral neck for patients whose individual risk for occult disease in that side is low. In turn, this may substantially reduce the toxicity from irradiation and the associated decrease in quality of life.

In the following, we begin by inspecting the dataset the model is built upon (@sec-data) and from it formulate some properties we require the bilateral model extension to have (@sec-requirements).

After that, in @sec-unilateral, we briefly summarize the original model formalism before we extend it to the contralateral side starting in @sec-ext-to-contra.

Next, we describe how the model training and the computational experiments were set up in @sec-methods and present the results in @sec-results, followed by a closing discussion (@sec-discussion).

# Data on Lymphatic Progression Patterns

Originally, the only available data of detailed - i.e. per patient and per LNL - lymphatic involvement was a reconstructed table from the findings in @sanguineti_defining_2009, which included only ipsilateral nodal involvement for the LNLs I to IV.

To be able to create models for lymphatic tumor progression that include more LNLs and the contralateral side, we have collected a detailed dataset of 833<!-- num patients --> patients with newly diagnosed oropharyngeal squamous cell carcinomas. It reports the involvement of every patient individually and per lymph node level in tabular form, in addition to other clinico-pathological information such as age, T-category, and HPV p16 status.

Their patient records have been collected at four different institutions and a brief overview over some of their patients’ characteristics are shown in @tbl-data-overview. Note that the data from the Inselspital Bern and the Centre Léon Bérard only consists of patients treated with some form of neck dissection. Since this treatment is more commonly chosen for early T-category patients, they also make up a larger portion of the respective dataset.

In [4]:
from typing import Literal

def align(
  column: pd.Series,
  where: Literal["right", "left", "center"],
) -> list[str]:
  """Align a column."""
  return [f"text-align: {where}"] * len(column)

def right_align(column: pd.Series) -> list[str]:
  """Right align column."""
  return align(column, where="right")

def highlight(column: pd.Series, mapping: dict) -> list[str]:
  """Color based on `mapping`."""
  colors = column.map(mapping)
  return colors.map(lambda x: f"color: {x}")

def highlight_bool(
  column: pd.Series,
  c_false: str = COLORS["green"],
  c_true: str = COLORS["red"],
) -> list[str]:
  """Make cell read (`False`) or green (`True`)."""
  return highlight(column, mapping={True: c_true, False: c_false})

def highlight_t_stage(
  column: pd.Series,
  c_early: str = COLORS["green"],
  c_late: str = COLORS["red"],
) -> list[str]:
  """Highlight `early` and `late`."""
  is_early = column == "early"
  return highlight_bool(is_early, c_false=c_late, c_true=c_early)


col_map = {
  COL.inst: "Institution",
  COL.age: "Age",
  COL.nd: "Neck Dissection",
  COL.t_stage: "T-category",
  COL.n_stage: "N-category",
}
short_col_map = {tpl[-1]: val for tpl, val in col_map.items()}

raw = utils.load_patient_data(paths.data)
subdata = raw[col_map.keys()]
subdata.columns = subdata.columns.get_level_values(2)
subdata = (
  subdata
  .rename(columns=short_col_map)
  .reset_index(drop=True)
)

def n0_mean(n_stage: pd.Series) -> float:
  """Compute portion of N0 patients."""
  return (n_stage == 0).mean()

def early_mean(t_stage: pd.Series) -> float:
  """Compute early T-category portion."""
  return t_stage.isin([0,1,2]).mean()

grouped = (
  raw.groupby(
    by=COL.inst,
  ).aggregate(**{
    "Total": pd.NamedAgg(column=COL.age, aggfunc="count"),
    "Age (median)": pd.NamedAgg(column=COL.age, aggfunc="median"),
    "Neck Dissection": pd.NamedAgg(column=COL.nd, aggfunc="mean"),
    "N0": pd.NamedAgg(column=COL.n_stage, aggfunc=n0_mean),
    "Early T-Cat.": pd.NamedAgg(column=COL.t_stage, aggfunc=early_mean),
    "Mid. Ext.": pd.NamedAgg(column=COL.midext, aggfunc="mean"),
  }).convert_dtypes()
)
grouped.index.name = "Institution"
(
  grouped
  .reset_index()
  .style
  .format(
    formatter="{:>.0%}",
    subset=["Neck Dissection", "N0", "Early T-Cat.", "Mid. Ext."],
  )
  .apply(
    func=right_align,
    axis="index",
    subset=grouped.columns[0:],
  )
  .hide()
)

## Contralateral Involvement Prevalence

These datasets allow us to investigate correlations between the involvement of individual LNLs, or between risk factors and patterns of involvement. In @fig-data-strat, we have plotted the prevalence of each contralateral LNL’s involvement, stratified by T-category, ipsilateral number of involved LNLs, and whether the tumor extended over the mid-sagittal line. A similar but more complete stratification is also tabulated in the appendix in @tbl-data-strat.



The left panel in @fig-data-strat indicates that T-category is correlated with contralateral involvement (as it is with overal involvement). This is simply because T-category may on average be considered a surrogate for the time between onset of disease and diagnosis. I.e., a patient with a T4 tumor was – on average – diagnosed later than a patient with a T1 tumor. Thus, the former did have more time to develop metastases.

Similarly, ipsilateral involvement correlates with contralateral metastasis. The tumor of a patients with many metastases in ipsilateral LNLs was probably able to spread for longer (or faster) compared to a tumor in a patient with no nodal disease. This, too, may therefore be considered a surrogate for the duration of the disease. In rare cases, bulky nodal disease ipsilaterally may also redirect lymph fluids to the contralateral side. <!-- citation needed? -->

Lastly, the right panel in @fig-data-strat shows that patients with a tumor crossing the mid-sagittal line show contralateral involvement vastly more often compared to patients with clearly lateralized tumors. This makes intutitive sense, because the lymphatic system in the head and neck region is typically symmetric and thus no major vessels cross the midline. Therefore, interstitial fluids from the primary tumor – which we assume to carry living malignant cells – may only reach the blind-ended lymphatic vessels in the contralateral neck via short-ranged diffusion. Which in turn is only possible when the primary tumor is close enough to the mid-sagittal line or crosses it. <!-- citation(s) needed? or is this intuition good enough? -->

## Requirements for a Bilateral Model

Based on the observations of the [previous section](#sec-data-strat), any potential model that aims to also predict the risk for contralateral nodal involvement, should be able to take the following into account:

1.  More advanced T-category should lead to higher risk for nodal disease. One approach to achieve this via the expected time of diagnosis has already been developed in the form of a hidden Markov model \[@ludwig_hidden_2021\].
2.  The degree of ipsilateral involvement should give the model information on the time that may have passed between onset and diagnosis of the disease. This should come in addition to what can be inferred about this time from T-category alone.
3.  A tumor that extends over the mid-sagittal line should yield contralateral metastases with much higher probability.

Over the course of this work, we will first briefly recap the mentioned HMM in @sec-unilateral, which was so far used to model ipsilateral lymphatic progression only. Then, we intuitively extend it to include the contralateral side as well in @sec-ext-to-contra. In this section, we also introduce a way of modelling the tumor’s midline extension as a random variable (@sec-midline) and lastly talk about how it may affect the contralateral spread in @sec-params-symmetry.

<!-- TODO: Hint at methods and results... -->

## Data Availability

The entire data, including additional patients with tumors in other primary locations than the oropharynx, is publicly available: It may be [downloaded from LyProX](https://lyprox.org/patients/dataset) where it can be interactively explored too, [from GitHub](https://github.com/rmnldwg/lydata), [from zenodo](https://zenodo.org/search?q=lydata), or via the *Data-in-Brief* publications @ludwig_dataset_2022 and @ludwig_multi-centric_2023. Although these publications do not include the most recent dataset addition from Vall d’Hebron Barcelona Hospital.

# Unilateral Model for Lymphatic Progression

{ width="40%"}

Our first model to predict the lymphatic progression of HNSCC was introduced using Bayesian networks \[@pouymayou_bayesian_2019\]. We subsequently extended this work to a hidden Markov model (HMM) \[@ludwig_hidden_2021\] to allow an intuitive inclusion of T-category into the predictions. We will briefly summarize this HMM’s formalism before building on it to include the contralateral spread in @sec-ext-to-contra.

We model a patient’s state of involvement at an abstract time-step $t$ as a vector of hidden binary random variables:

$$
\mathbf{X}[t] = \begin{pmatrix} X_v[t] \end{pmatrix} \qquad v \in \left\{ 1, 2, \ldots, V \right\}
$$ {#eq-state-def}

Here, $V$ is the number of LNLs the model considers. The values a LNL’s hidden binary RV may take on are $X_v[t] = 0$ (`False`), meaning the LNL $v$ is healthy or free of metastatic disease, or $X_v[t] = 1$ (`True`), corresponding to some for of tumor presence (i.e., occult or clinical).

Since the state vector $\mathbf{X}[t]$ is $V$-dimensional and binary, there are $2^V$ distinct possible lymphatic involvement patterns, which we enumerate from $\boldsymbol{\xi}_0 = \begin{pmatrix} 0 & 0 & \cdots & 0 \end{pmatrix}$ to $\boldsymbol{\xi}_{2^V} = \begin{pmatrix} 1 & 1 & \cdots & 1 \end{pmatrix}$.

Any hidden Markov model is fully described by three quantities:

1.  A starting state $\mathbf{X}[t=0]$ at time $t=0$ just before the patient’s tumor formed. In our case, this is always the state where all LNLs are still healthy $\boldsymbol{\xi}_0$.
2.  The *transition matrix* $$
    \mathbf{A} = \left( A_{ij} \right) = \big( P \left( \mathbf{X}[t+1] = \boldsymbol{\xi}_j \mid \mathbf{X}[t] = \boldsymbol{\xi}_i \right) \big)
    $$ {#eq-trans-matrix} where the value at row $i$ and column $j$ represents the probability to transition from state $\boldsymbol{\xi}_i$ to $\boldsymbol{\xi}_j$ during the time-step from $t$ to $t+1$. Note that we prohibit self-healing, meaning that during a transition, no LNL may change their state from $X_v[t]=1$ to $X_v[t+1]=0$. This effectively masks large parts of the transition matrix to be zero.
3.  Lastly, the *observation matrix* $$
    \mathbf{B} = \left( B_{ij} \right) = \big( P \left( \mathbf{Z} = \boldsymbol{\zeta}_j \mid \mathbf{X}[t_D] = \boldsymbol{\xi}_i \right) \big)
    $$ {#eq-obs-matrix} where in row $i$ and at column $j$ we find the probability to *observe* a lymphatic involvement pattern $\mathbf{Z} = \boldsymbol{\zeta}_j$, given that the true (but hidden) state of involvement at the time of diagnosis $t_D$ is $\mathbf{X}[t_D] = \boldsymbol{\xi}_i$.

Note that the transition matrix $\mathbf{A}$ is parametrized using the spread probabilities of a directed acyclic graph (DAG) that we define as the underlying mechanistic representation of the lymphatic network. An example of such a DAG is shown in @fig-small-graph.

Using the introduced quantities, we can evolve the distribution of all possible hidden states from $\mathbf{X}[t=0] = \boldsymbol{\xi}_0$ step by step, by successively multiplying this vector with the transition matrix $\mathbf{A}$. At the time of the diagnosis $t_D$, we multiply the result with the observation matrix $\mathbf{B}$. We may then look up the likelihood of a patient presenting with the diagnosis $\mathbf{Z}=\boldsymbol{\zeta}_i$ in the $i$-th entry of the final result.

However, the remaining issue is that the value of $t_D$ is unknown, i.e. over how many time-steps the HMM should be evolved. We solve this problem by marginalizing over the time of diagnosis. Different distributions over the diagnosis times can then be choosen based on T-category. For instance, the mean of the time-prior to marginalize over the diagnosis time for early T-category patients $P\left( t_D \mid \text{early} \right)$ may be shifted towards earlier times than the one for advanced T-category patients $P\left( t_D \mid \text{early} \right)$. This gives us for example

$$
P\left( \mathbf{X} \mid \text{T}x = \text{early} \right) = \sum_{t=0}^{t_\text{max}} P \left( \mathbf{X} \mid t \right) \cdot P(t \mid \text{early})
$$

For later use, we define at this point the matrices of the dsitributions over all hidden states, given all time-steps $\boldsymbol{\Lambda}$:

$$
\boldsymbol{\Lambda} = P \left( \mathbf{X} \mid \mathbf{t} \right) = \begin{pmatrix}
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^0 \\
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^1 \\
\vdots \\
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^{t_\text{max}} \\
\end{pmatrix}
$$ {#eq-lambda-matrix}

Where the $k$-th row in this matrix corresponds to the distribution over hidden states after $t=k-1$ time-steps.

In this work, we use binomial distributions $\mathfrak{B} \left( t_D, p_{\text{T}x} \right)$ as time-priors which have one free parameter $p_{\text{T}x}$ for each group of patients we differentiate based on T-category. Also, we fix $t_\text{max} = 10$, which means that the expected number of time-steps from the onset of a patient’s disease to their diagnosis is $\mathbb{E}\left[ t_D \right] = 10 \cdot p_{\text{T}x}$.

## Likelihood Function

With the formalism introduced above, we can write the likelihood function for a patient to present with a diagnosis consisting of an observed state and a T-category $d = \left( \boldsymbol{\zeta}_i, \text{T}x \right)$ as follows:

$$
\ell = P \left( \mathbf{Z} = \boldsymbol{\zeta}_i \mid \text{T}x \right) = \sum_{t=0}^{t_\text{max}} \left[ \boldsymbol{\xi}_0 \cdot \mathbf{A} \cdot \mathbf{B} \right]_i \cdot P \left( t \mid \text{T}x \right)
$$ {#eq-single-patient-llh}

Above, the quantity inside $\left[ \ldots \right]_i$ denotes the $i$-th component of the vector that is the result of the vector and matrix multiplications in the square brackets. Note that it is also possible to account for missing involvement information: If a diagnosis (like fine needle aspiration (FNA)) is only available for a subset of all LNLs, we can sum over all those possible complete observed states $\boldsymbol{\zeta}_j$ that match the provided diagnosis.

The single-patient likelihood $\ell$ in @eq-single-patient-llh depends on the spread parameters shown in @fig-small-graph via the transition matrix $\mathbf{A}$ and on the binomial parameters $p_{\text{T}x}$ via time-priors. In this work, we will only differentiate between “early” (T1 & T2) and “advanced” (T3 & T4) T-categories. Therefore, the parameter space of the unilateral model is:

$$
\boldsymbol{\theta} = \left( \left\{ b_v \right\}, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.} \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
$$ {#eq-param-space}

And it is our goal to infer the values of these parameters for a given dataset $\mathcal{D} = \left( d_1, d_2, \ldots, d_N \right)$ of OPSCC patients. The likelihood of these $N$ diagnoses is simply the product of their individual likelihoods as defined in @eq-single-patient-llh. For numerical reasons, we typically compute the data likelihood in log space:

$$
\log \mathcal{L} \left( \mathcal{D} \mid \boldsymbol{\theta} \right) = \sum_{i=1}^N \log \ell_i
$$ {#eq-log-likelihood}

The methodology we use to infer the model’s parameters is detailed in @sec-sampling.

## Model Prediction in the Bayesian Context

Our stated goal is to compute the risk for a patient’s true nodal involvement state $\mathbf{X}$, *given* their individual diagnosis $d = \left( \boldsymbol{\zeta}_k, \text{T}x \right)$. Using Bayes’ law, this is written as:

$$
P \big( \mathbf{X} \mid \mathbf{Z}=\boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x \big) = \frac{P \left( \boldsymbol{\zeta}_k \mid \mathbf{X} \right) P \big( \mathbf{X} \mid \boldsymbol{\hat{\theta}}, \text{T}x \big)}{\sum_{i=0}^{2^V} P \left( \boldsymbol{\zeta}_k \mid \mathbf{X}=\boldsymbol{\xi}_i \right) P \big( \mathbf{X}=\boldsymbol{\xi}_i \mid \boldsymbol{\hat{\theta}}, \text{T}x \big)}
$$ {#eq-uni-bayes-law}

The term $P \left( \boldsymbol{\zeta}_k \mid \mathbf{X} \right)$ is defined solely by sensitivity and specificity of the diagnostic modality. Terms like this already appeared in the definition of the observation matrx in @eq-obs-matrix. The *prior* $P \big( \mathbf{X} \mid \boldsymbol{\hat{\theta}} \big)$ in the above equation is the crucial term that is supplied by a trained model and its parameters $\boldsymbol{\hat{\theta}}$.

It is possible to compute this *posterior* probability of true involvement not only for one fully defined state $\mathbf{X}$, but also for e.g. individual LNLs: For example, the risk for involvement in level IV would be a marginalization over all states $\boldsymbol{\xi}_i$, where $\xi_{i4}=1$. Formally:

$$
P \big( \text{IV} \mid \mathbf{Z}=\boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x  \big) = \sum_{k \, : \, \xi_{k4}=1} P \big( \mathbf{X} = \boldsymbol{\xi}_k \mid \boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x  \big)
$$ {#eq-marg-over-posterior}

# Extension to a Bilateral Model

A naive approach to model the contralateral lymphatic spread would be to simply employ two independent unilateral models as introduced in @sec-unilateral. During training, one could even enforce some shared parameters between these two models, like the parameterization of the distributions over diagnose times or the spread among the LNLs. Additionally, we could think of an approach to incorporate the primary tumor’s mid-sagittal extension as a risk factor.

However, this approach lacks a way to describe the correlation between ipsi- and contralateral involvement. This is displayed in @tbl-data-strat and shows how often the contralateral LNLs I, II, III, and IV were involved, given all possible combinations of midline extension, T-category, and ipsilateral LNL III involvement. Unsurprisingly, the prevalence for contralateral involvement is consistently higher when the tumor extends over the mid-sagittal line or is of later T-category. But it is also more frequent when the ipsilateral side shows more severe involvement, which is here shown via the surrogate LNL III.

Thus, we attempt to extend the formalism in @sec-unilateral in such a way that the model’s ipsi- and contralateral side evolve synchronously. To achieve that, we start by writing down the posterior distribution of involvement an analogy to @eq-uni-bayes-law, which is now a joint probability of an involvement $\mathbf{X}^\text{i}$ ipsilaterally *and* an involvement $\mathbf{X}^\text{c}$ contralaterally, given a diagnosis of the ipsilateral LNLs $\mathbf{Z}^\text{i}$ and of the contralateral ones $\mathbf{Z}^\text{c}$:

$$
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right) = \frac{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right)}{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right)}
$$ {#eq-bilateral-bayes}

For the sake of brevity, we omit the dependency on the parameters and the T-category here.

The likelihood of the diagnoses given a hidden state simply factorise: $P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) = P \left( \mathbf{Z}^\text{i} \mid \mathbf{X}^\text{i} \right) \cdot P \left( \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{c} \right)$. And the two factors are contained in the observation matrices $\mathbf{B}^\text{i}$ and $\mathbf{B}^\text{c}$.

The term representing the model’s prior probability of hidden involvement does not factorize. However, if we assume that lymphatic spread typically does not cross the mid-sagittal line, we can write it as a factorising sum:

$$
\begin{aligned}
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) &= \sum_{t=0}^{t_\text{max}} P(t) \cdot P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid t \right) \\
&= \sum_{t=0}^{t_\text{max}} P(t) \cdot P \left( \mathbf{X}^\text{i} \mid t \right) \cdot P \left( \mathbf{X}^\text{c} \mid t \right)
\end{aligned}
$$ {#eq-bilateral-marginal}

This assumption makes intuitive sense: The two sides of the lymphatic network in a typical patient are approximately mirror images of each other. Thus, no major vessels cross the mid-sagittal line. There may, however, be diffusion of lymph fluid accross this line or bulky involvement that redirects lymphatic drainage significantly.

Using this assumption along with @eq-lambda-matrix, we can write the above distribution algebraically as a product:

$$
P \left( \mathbf{X}^\text{i} = \boldsymbol{\xi}_n, \mathbf{X}^\text{c} = \boldsymbol{\xi}_m \right) = \left[ \boldsymbol{\Lambda}^\intercal_\text{i} \cdot \operatorname{diag} P(\mathbf{t}) \cdot \boldsymbol{\Lambda}_\text{c} \right]_{n,m}
$$ {#eq-bilateral-marginal-algebra}

## Parameter Symmetries

In general, the matrices $\boldsymbol{\Lambda}_\text{i}$ and $\boldsymbol{\Lambda}_\text{c}$ could be parameterized using a disjoint set of parameters. I.e., the ipsi- and contralateral spread rates are entirely different. However, using two sensible assumptions, we can reduce the parameter space by sharing some parameters between the sides:

1.  We assume the spread *among* the LNLs to be same on both sides. It is reasonable to assume the lymphatic system is symmetric. Thus, the spread rates from one LNL to the other should be symmetric, too. Formally, this means  
    $$
    \begin{aligned}
    b_v^\text{c} &\neq b_v^\text{i} \\
    t_{rv}^\text{c} &= t_{rv}^\text{i}
    \end{aligned}
    $$ {#eq-symmetries} for all $v \leq V$ and $r \in \operatorname{pa}(v)$.
2.  The tumor’s spread to the contralateral side in case of an extension over the midline is larger than if it was clearly lateralized, but smaller than its spread to the ipsilateral side. This assumption stems from a simple thought experiment: Consider moving the tumor from a clearly lateralized position accross the mid-sagittal plane to the same position, but on the contralateral side. In the beginning we would have $b_v^\text{c} < b_v^\text{i}$, while in the end, the situation is reversed. If a tumor extends over the mid-sagittal line, its contralateral spread rate can be expected to be in between these two extremes. We encode this in a *mixing parameter* $\alpha$ that captures a “degree of asymmetry”:  
    $$
    b_v^{\text{c},\epsilon=\texttt{True}} = \alpha \cdot b_v^\text{i} + (1 - \alpha) \cdot b_v^{\text{c},\epsilon=\texttt{False}}
    $$ {#eq-mixing} This means the model now uses three different sets of parameters to describe the spread from the tumor to the LNLs: $b^\text{i}_v$ for the spread to the ipsilateral LNLs, $b_v^{\text{c},\epsilon=\texttt{False}}$ for the spread to the contralateral LNLs as long as the tumor is clearly lateralized, and finally $b_v^{\text{c},\epsilon=\texttt{True}}$ when it crosses the midline. Note, however, that these three sets of spread rates only account for $2 \cdot 2^V + 1$ parameters, since they are coupled via the mixing parameter $\alpha$.

Our parameter space has now expanded to

$$
\boldsymbol{\theta} = \left( \left\{ b_v^\text{i} \right\}, \left\{ b_v^\text{c} \right\}, \alpha, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.} \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
$$ {#eq-bi-param-space}

which is less than double in size, compared to the unilateral model. From these parameters, we now build three different transition matrices: The unchanged $\mathbf{A}_\text{i}$ for the ipsilateral side, one $\mathbf{A}_\text{c}^{\epsilon=\texttt{False}}$ for the contralateral side that covers the progression as long as the tumor is lateralized, and $\mathbf{A}_\text{c}^{\epsilon=\texttt{True}}$ in case of a tumor that has crossed the mid-sagittal line.

## Modelling Midline Extension

To account for the increased prevalence of involvement on the contralateral side when the tumor is not clearly lateralized anymore, we also model the tumor’s extension over the mid-sagittal line as a binary random variable. It starts lateralized and at every time-step there is a finite probability $p_\epsilon$ that the tumor grows over the symmetry plane of the patient.

The overall probabilities to find a patient with a clearly lateralized tumor or one that extends over the mid-sagittal line after $t$ time-steps are then respectively given by

$$
\begin{aligned}
P(\epsilon = \texttt{False} \mid t) &= (1 - p_\epsilon)^t \\
P(\epsilon = \texttt{True} \mid t) &= 1 - P(\epsilon = \texttt{False} \mid t)
\end{aligned}
$$

In @fig-model-midext-evo, we visualize how the prior distribution over diagnose times $P(t)$, the conditional probability of midline extension $P(\epsilon \mid t)$, and their joint $P(\epsilon, t)$ evolve over the course of a patient evolution.

{ width="50%"}

Using this, it is straightforward to write down the matrix of state distributions for all time-steps, as in @eq-lambda-matrix covering the contralateral hidden state evolution:

$$
\boldsymbol{\Lambda}_\text{c}^{\epsilon=\texttt{False}} =
\left(
\begin{array}{r}
\boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{0\phantom{t_\text{max}}} \\
(1-p_\epsilon) \cdot \boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{1\phantom{t_\text{max}}} \\
\hfill \vdots \hfill \\
(1-p_\epsilon)^{t_\text{max}} \cdot \boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{t_\text{max}\phantom{0}} \\
\end{array}
\right)
$$

where we used the transition matrix $\mathbf{A}_\text{c}^{\epsilon=\texttt{False}}$ that depends on the base spread parameters $b_v^{\text{c},\epsilon=\texttt{False}}$.

The case when midline extension is eventually present is more complicated: We already marginalize over the exact time-step when the tumor grows over the mid-sagittal line. But whenever that happens, we also need to change the contralateral transition matrix to use the increased spread rates $b_v^{\text{c}, \epsilon=\texttt{True}}$ from the tumor to the contralateral LNLs, given by the linear mix in @eq-mixing. We can achieve the correct marginalization by iteratively building the joint $P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid t \right)$. We start from $t=0$, where we know that all LNLs are healthy - i.e., the contralateral neck is in the state $\mathbf{X}_\text{c}=\boldsymbol{\xi}_0$ - and the tumor is lateralized ($\epsilon=\texttt{False}$):

$$
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{False} \mid t=0 \right) = \boldsymbol{\xi}_0
$$

Then we consider an arbitrary later time-step $t=\tau+1$. There are two possible scenarios we need to marginalize over:

1.  The tumor was still lateralized at $t=\tau$ and just grew over the midline. Under this scenario, the probability for having a midline extension at $t=\tau+1$ is given by $p_\epsilon$. We use this to weight the contralateral state distribution that was so far evolved without increased contralateral spread.
2.  The mid-sagittal line was already crossed by the tumor. In that case, the probability is 1 to remain in that lateralization state. Thus, for this case we simply add the same distribution we are computing, but from one time-step earlier, arriving at a recursion relation:

$$
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid \tau + 1 \right) = \big[ p_\epsilon P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{False} \mid \tau \right) + P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid \tau \right) \big]^\top \cdot \mathbf{A}_\text{c}^{\epsilon=\texttt{True}}
$$

We can collect the iteratively computed distributions for the midline extension case to define the matrix over the states given all time-steps, in analogy to @eq-lambda-matrix:

$$
\boldsymbol{\Lambda}_\text{c}^{\epsilon=\texttt{True}} = \begin{pmatrix}
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid 0 \right) \\
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid 1 \right) \\
\vdots \\
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid t_\text{max} \right) \\
\end{pmatrix}
$$

Using this, we can again write the joint of ipsi- and contralateral involvement - now also for the case of mid-sagittal extension - algebraically as before in @eq-bilateral-marginal-algebra:

$$
P \left( \mathbf{X}^\text{i} = \boldsymbol{\xi}_n, \mathbf{X}^\text{c} = \boldsymbol{\xi}_m, \epsilon \right) = \left[ \boldsymbol{\Lambda}^\intercal_\text{i} \cdot \operatorname{diag} P(\mathbf{t}) \cdot \boldsymbol{\Lambda}_\text{c}^\epsilon \right]_{n,m}
$$

We can use this term to compute the likelihood of all patients with and without midline extension separately. And if for some patients the information of tumor lateralization is not available, we can simply sum the above term once for $\epsilon=\texttt{False}$ and once for $\epsilon=\texttt{True}$ to marginalize over the unknown variable.

The final parameter space of our extended model has now reached this size:

$$
\boldsymbol{\theta} = \left( \left\{ b_v^\text{i} \right\}, \left\{ b_v^\text{c} \right\}, \alpha, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.}, p_\epsilon \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
$$ {#eq-ext-param-space}

In @fig-model-state-dist we plot the state distribution for a full midline model as two separate heatmaps. To keep it manageable to interpret, this example model only considers the LNLs II, III, and IV ipsi- and contralaterally, resulting in $2 \times 2^3 \times 2^3 = 128$ distinct states.



# Methods

<!-- Maybe I should break this apart into separate chapters on data, sampling, etc. -->

In this section, we detail how the experiments were performed. Every figure, table, and result is fully reproducible via the GitHub repository [`rmnldwg/bilateral-paper`](https://github.com/rmnldwg/bilateral-paper). It also contains the raw manuscript and instructions on how to recreate all figures, tables, and the final document.

## Involvement Data Consensus

It is possible to provide our model with multiple different diagnostic modalities, each being characterized by different pairs of sensitivity and specificity. However, we instead chose to combine them into a single “consensus” diagnosis before parameter inference. We opted for this because the literature values of sensitivity and specificity \[@de_bondt_detection_2007;@kyzas_18f-fluorodeoxyglucose_2008\] of imaging modalities like MRI and CT do not plausibly match some of our observations: In the USZ cohort, 78% of OPSCC patients where diagnosed with ipsilateral LNL II involvement via diagnostic imaging. This is virtually impossible with sensitivities around 80% and specificities lower than 100%.

Our pre-training consensus was formed by considering all reported diagnostic information for a particular patient and LNL. When conflicts arose, we computed the *most likely* true state of involvement using the literature sensitivity and specificity values \[@de_bondt_detection_2007;@kyzas_18f-fluorodeoxyglucose_2008\].

## MCMC Sampling

For parameter inference, we used the Python package [`emcee`](https://emcee.readthedocs.io/en/stable/) \[@foreman-mackey_emcee_2013\]. It implements efficient MCMC sampling algorithms that employ multiple parallel samplers for affine invariance and better performance on multi-core CPUs. The sample proposal algirothms used by us are based on differential evolution moves \[@ter_braak_differential_2008;@nelson_run_2013\]. The [`emcee`](https://emcee.readthedocs.io/en/stable/) library was provided with the likelihood implemented by our [`lymph-model`](https://lymph-model.readthedocs.io/en/stable/) Python package.

For each dimension in the parameter space of the model, we initialized 12 of these parallel samplers, called “walkers”, with random values in the unit cube. Every time all of these walkers advanced 50 steps, the autocorrelation time of the chains was estimated. For short chains, this estimate is not trustworthy, but stabilizes for longer chains. We therefore considered a sampling to be converged when two criteria where met:

1.  The change in the autocorrelation time was less then 5.0e-2.
2.  The estimate of the autocorrelation dropped below $n$ / 50 where $n$ is the length of the chain up to that point.

All samples up to this convergence - called the *burn-in phase* - were discarded. We only kept another 10 samples after that, which were spaced 10 steps apart.

## Computing the Observed and Predicted Prevalence of Involvement Patterns

We want to assess the model’s capability to approximate the distribution of lymphatic involvement patterns seen in the data. To that end, we compare the prevalence of some invovlement patterns under selected scenarios with the model’s prediction for how often these involvements it expects to see, given these scenarios.

In this context, a “scenario” includes the patient’s T-category $\text{T}x$ and whether the patient’s tumor extended over the mid-sagittal line, i.e. $\epsilon=\texttt{True}$ or $\epsilon=\texttt{False}$.

An involvement pattern specifies for each ipsi- and contralateral LNL whether it is “healthy”, “involved”, or “masked”. If it is “masked”, we essentially state that we are not interested in the involvement of that LNL and the prevalence will be marginalized over this LNL’s involvement.

For example, we may be interested in the prevalence of contralateral LNL II involvement (i.e., contra LNL II “involved” and all other LNLs “masked”) under the scenario of early T-category (T0-T2) and no midline extension ($\epsilon=\texttt{False}$). To compute this prevalence in the data, we select all patients of this scenario (in our data, this amounts to 379<!-- TODO: dyamically compute this --> patients). Of those, 28<!-- TODO: dynamically compute this --> were found to harbor metastases in their contralateral LNL II. Therefore, the prevalence is 7%<!-- TODO: dynamically compute this -->.

When displaying this data prevalence, we often choose to draw a *beta posterior* over the “true” prevalence, hinting at the fact that our data merely represents a limited sample. The beta posterior follows from a uniform beta distribution as prior and a binomial likelihood for the number of patients with the involvement of interest, given the parameer for the “true” prevalence. The resulting distribution has its maximum at the observed prevalence, but in addition gives a visual intuition for the variance of of the observed quantity. I.e., when we observe 3 out of 10 events, the beta posterior is much wider than if we observe 300 out of 1000 for he same prevalence. It also allows us to check not only if the model is accurate, but also whether it reflects the uncertainty contained in the data.

Predicting the prevalence using our model amounts to computing the following probability:

$$
P \left( \text{II}^\text{c} \mid \epsilon=\texttt{False}, \text{T}x=\text{early} \right) = \frac{P \left( \text{II}^\text{c}, \epsilon=\texttt{False} \mid \text{T}x=\text{early} \right)}{P \left( \epsilon=\texttt{False} \mid \text{T}x=\text{early} \right)}
$$

In the enumator, we marginalize over all ipsi- and contralateral LNLs’ involvements, except for LNL II contralaterally. This is similar to the marginalization in @eq-marg-over-posterior, although we are summing over different quantities. In the denominator, we can simply insert the joint distribution over midline extension and diagnose time $P \left( \epsilon, t \right)$ marginalized over $t$ using the early T-category’s time-prior.

Since we compare it to the data, which does not report true but only observed involvement – although pathologically investigated LNLs may be as close as possible to the ground truth – we do not consider posteriors of the form $P \left( \mathbf{X} \mid \mathbf{Z} \right)$ here. Instead, we compute probabilities of observed involvement $P \left( \mathbf{Z} \right)$, as in the likelihood @eq-single-patient-llh.

When plotted, we usually display histograms over the model’s predictions. Each of their values was computed from a different parameter set drawn during MCMC sampling, effectively giving us a distribution over the prevalences. Ideally, the histograms approximate the location and width of the Beta posteriors when attempting to describe the data they were trained on.

Note that we decided to omit the y-axis ticks and labels in these figures over prevalences and risks. The y-axis in these plots measures the probability density and its numerical values are not intuitively interpretable. Instead, we occasionally use the freed space to label e.g. rows of subplots.

# Results

First, in @fig-model-burnin-history, we verify the sampling converged successfully by inspecting two monitoring quantities: The autocorrelation time of the MCMC chain and the acceptance fractions of the parallel walkers.



In @tbl-midline-params, we tabulate the mean and standard deviation of the sampled parameters for the full midline model.

In [5]:
def map_param_names(name: str) -> str:
  """Make parameter names more readable."""
  try:
    return {
      "midext_prob": "Mid. ext. probability",
      "mixing": "Mixing ⍺",
      "late_p": "late T-cat. binom. prob.",
    }[name]
  except KeyError:
    return re.sub(
      r"(ipsi|contra)_", r"\g<1>: ", name
    ).replace(
      "to", " ➜ "
    ).replace(
      "_spread", ""
    )

model = shared.get_model(which="full", load_samples=True)
samples = shared.get_samples(which="full")

names = [map_param_names(p) for p in model.get_params()]
means, stds = samples.mean(axis=0), samples.std(axis=0)

early_midext_prob = model.state_dist(t_stage="early")[1].sum()
late_midext_prob = model.state_dist(t_stage="late")[1].sum()

params_table = pd.DataFrame({"Parameter": names, "Mean": means, "Std. Dev.": stds})
(
  params_table.style
  .format("{:.2%}", subset=["Mean"])
  .format("± {:.2%}", subset=["Std. Dev."])
  .apply(right_align, subset=["Mean", "Std. Dev."])
  .hide()
)

## Prevalence Predictions

We want to investigate whether and to what extent the model can fulfill the requirements laid out in @sec-requirements. To that end, we compare its predictions for contralateral involvement against observations in the data. This is done given scenarios that differ in T-category and/or midline extension and/or ipsilateral involvement.

### Dependence of Contralateral Involvement on T-Category and Midline Extension

In @fig-model-prevalences-overall, we plot the prevalence of contralateral involvement of the LNLs II, III, and IV for the four scenarios made up of the possible combinations of early and late T-category, as well as lateralized and midline extending tumors.



@Fig-model-prevalences-overall shows nicely how the model is capable of accurately taking the most important risk factors, i.e. T-category and midline extension, into account. As observed in the data, the model predicts that the prevalence of contralateral LNL II involvement jumps from below 8% <!-- TODO: make this dynamic --> for early T-category lateralized tumors to almost 40% <!-- TODO: make this dynamic --> when the tumor is of advanced T-category and crosses the mid-sagittal line.

However, for early T-category scenarios with midline extension, the model does seem to overestimate contralateral LNL II and III invovlement. This likely stems from the the small sample size of this relatively rare scenario as hinted at by the wide beta posteriors.

### Correlation between Ipsi- and Contralateral Involvement



In @fig-model-prevalences-with-ipsi we display the model’s ability to capture the correlation between ipsi- and contralateral involvement. It shows that the prevalence of metastases in the two sides of the neck is correlated via the time of diagnosis, despite the model not having any direct connections between the two side. However, there are some discrepancies in the model’s prediction: It cannot quite capture the correlation of ipsi- and contralateral involvement that is seen in the data. This is notable for example in the bottom right subplot where the prevalence is overestimated for the scenario of an (almost) healthy ipsilateral neck and underestimated when at least LNL III shows metastases. One reason for this may be the width of the time-prior: If it was wider, the posterior over the diagnosis time given the ipsilateral involvement could more flexibly shift to reflect a more severe contralateral involvement.

<!-- NOTE: This last sentence is purely speculative! We need to discuss this. -->

### Prevalence of Midline Extension

{ width="50%"}

Lastly, in @fig-model-prevalences-midext, we plot the prevalence of midline extension in the data versus our model’s prediction. It is obvious the model cannot match the large spread between early and advanced T-category seen in the data. This is because to achieve that, it would need to increase the advanced T-category patient’s prior distribution over diagnosis times and at the same time reduce the probability of the tumor to cross the midline during a time-step. But since the time-priors parameter is also coupled with the spread probabilities among the LNLs, the model does not have that freedom.

<!-- Should the paragraphs below be part of the discussion? -->

However, we do not consider this discrepancy a major limitation of the model: We will not realistically be interested in the probability of midline extension, as it is always possible to assess it with high certainty. That is also the reason why we initially modelled the midline extension *not* as a random variable, but as a global risk factor that would have been turned on or off from the onset of a patient’s disease evolution. This, however, lead to overly high risks for contralateral involvement in advanced T-category patients with midline extension, because then the model assumes an increased spread to the contralateral side from the onset of the disease. Which is probably not true in a majority of those cases. Thus, treating it as a random variable that only becomes true during a patient’s disease evolution resulted in a better description of the data.

Formally, the wrong prediction of midline extension prevalence makes little difference, since it is always given: Instead of $P\left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right)$, we typically compute $P\left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right)$, which does not suffer from the wrong probability of midline extension, as the distribution over hidden states is renormalized:

$$
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right) = \frac{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \right) P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \right)}{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right)}
$$

Note that a distribution over $\epsilon$ appears both in the enumerator and the denominator, which largely cancel each other, leaving only the midline extension’s effect on the distribution over hidden states in the prediction.

Also, the discrepancy in midline extension prevalence between early and advanced T-category is particularly pronounced in oropharyngeal SCC patients. For example, in oral cavity SCC, the midline extension only increases from 15.4% (20 out of 130) to 33.3% (13 out of 39).

## Prediction of Risk for Occult Disease

In this section, we investigate how the model may be applied clinically: We want to estimate the risk for occult disease in some or all LNLs, given the patient’s individual diagnosis. In terms of our model, this diagnosis consists of the T-category, the lateralization of the tumor (does it extend over the mid-sagittal line?), and which LNLs are clinically involved, e.g. because some lymph nodes appear enlarged in an MRI scan or show increased glycolytic activity on an FDG PET/CT scan.



@Fig-model-risks shows this predicted risk of occult disease in the contralateral LNLs II, III, and IV for interesting combinations of these three risk factors. This risk is computed *given* a CT diagnosis assessing the per-LNL clinical involvement for which we assumed a sensitivity of 81% and a specificity of 76% \[@de_bondt_detection_2007\].

### Contralateral LNL II

The variable impacting the prediction for contralateral level II involvement in our model is the tumor’s lateralization. For example, a patient with a clearly lateralized early T-category tumor and a clinically N0 neck is assigned a 1-2% <!-- TODO: make this dynamic --> risk for occult disease in contralateral LNL II. Under the same scenario, but *with* mid-sagittal extension, the risk jumps to almost 7%. <!-- TODO: make this dynamic -->

T-category plays a lesser role: Considering the scenario of a tumor that crosses the mid-sagittal line and an ipsilateral neck where at least LNL III is clinically involved, the risk for occult contralateral LNL II disease is around 8.5%. <!-- TODO: make this dynamic --> For the same scenario and an advanced T-category tumor, the risk increases to 11%. <!-- TODO: make this dynamic -->

Lastly, the predicted risk is also correlated via the time-steps to the degree of ipsilateral involvement. Changing the aforementioned scenario (advanced T-category, midline extension, ipsilateral LNL III clinically involved) to one where the patient presents with a clinically N0 neck, the risk for occult disease in the contralateral LNL II falls from 11% <!-- TODO: make this dynamic --> to 9.5%. <!-- TODO: make this dynamic -->

Taken together, T-category and ipsilateral involvement may still considerably impact the risk prediction for contralateral involvement: In @fig-model-risks the scenarios underlying the orange (6.5% <!-- TODO: make this dynamic -->) and the red (11% <!-- TODO: make this dynamic -->) histograms differ in T-category (early vs advanced) and clinical diagnosis (N0 vs ipsilateral at least LNL III involved).

### Contralalteral LNL III

As shown in the center subplot of @fig-model-risks, the risk for occult disease in the contralateral LNL III only ever crosses a 5% threshold if the upstream level II is involved too, resulting in a risk of 5% to 6%<!-- TODO: make dynamic -->. As long as the contralateral neck appears clinically node negative, however, even an advanced T-category and midline extension do not yield a risk higher than 2%.

### Contralateral LNL IV

Our model’s prediction for the contralateral LNL IV show that even in the most extreme case, with an advanced T-category tumor extending over the mid-sagittal line as well as metastases in every ipsilateral LNL and every contralateral upstream level, the risk does not exceed 5%

# Discussion

## Summary

In this work we present a formalism to model the ipsi- and contralateral lymphatic involvement of oropharyngeal SCC patients. An ipsilateral model has been previously developed and published \[@ludwig_hidden_2021;@ludwig_modelling_2023\]. Based on this, we introduce an extension that leaves the ipsilateral model untouched, but extends it to the contralateral side in an intuitive and comprehensible manner.

The model’s performance w.r.t. its ability to describe the data on contralateral nodal involvement was evaluated based on a dataset of 833<!-- TODO: make this dynamic --> patients from four institutions. To the best of our knowledge, this is the first time lymphatic tumor progression has been modelled in such detail and to such an extent. While there was work on regional tumor spread, these models were conceptually different, more limited in the LNLs for which they make predictions, and not trained with real patient data \[@benson_markov_2006;@jung_development_2017\].

The model takes the clincal diagnosis of the lymph node levels, the primary tumor’s T-category, as well as lateralization into account when predicting the personalized risk for occult disease in any LNL of interest. Owing to the relatively few parameters, the model is highly interpretable and every parameter can be intuitively explained, while still offering good accuracy in its predictions.

## Implications for Contralateral Elective Nodal Treatment

The predictions of this model are being used to inform a clinical trial on volume deescalation at the University Hospital Zurich \[citation needed<!-- TODO: add -->\]. When accepting a 5% risk of occult disease in any given LNL, the model suggests the following contralateral elective irradiation (assuming the respective LNL appears clinically healthy):

-   LNL II when the tumor extends over the mid-sagittal line
-   LNL III when LNL II is clinically, the tumor is of advanced T-category, crosses the midline, and the ipsilateral neck shows involvement of at least LNL II
-   LNL IV only for severe ipsilateral involvement, advanced T-category with obvious midline extension and contralateral upstream involvement

# Acknowledgement

This work was supported by:

-   the Clinical Research Priority Program “Artificial Intelligence in Oncological Imaging” of the University of Zurich
-   the Swiss Cancer Research Foundation under grant number KFS 5645-08-2022

# Contralateral Prevalence of Involvement

In [6]:
lnl_cols = get_lnl_cols("contra", lnls=["I", "II", "III", "IV"])
num_ipsi_inv = raw[get_lnl_cols("ipsi")].sum(axis="columns")

contra_inv = raw[lnl_cols].copy()
contra_inv.columns = contra_inv.columns.droplevel([0,1])
contra_inv["t_stage"] = raw[COL.t_stage].apply(lambda x: "early" if x <= 2 else "advanced")
contra_inv["ipsi"] = num_ipsi_inv.map(lambda x: str(x) if x <= 1 else "≥ 2")
contra_inv["midext"] = raw[COL.midext]

grouped = contra_inv.groupby(by=["t_stage", "ipsi", "midext"])
num_involved = grouped.sum()
total = grouped.count()
percent_involved = num_involved / total
idx = total.index.rename(["T-cat.", "ipsi", "Mid. ext."])

total.index = idx
num_involved.index = idx
percent_involved.index = idx

involved = num_involved.join(
  100 * percent_involved,
  rsuffix=" (%)",
).sort_index(axis="columns")
involved.columns = pd.MultiIndex.from_product(
  [["I", "II", "III", "IV"],
   ["n", "%"]],
  names=["LNL", ""]
)
involved["total", "n"] = total["I"]
(
  involved
  .reset_index()
  .sort_values("ipsi", ascending=True)
  .sort_values("T-cat.", ascending=False)
  .style
  .format(precision=2)
  .apply(right_align, subset=involved.columns[0:])
  .apply(highlight_t_stage, subset=["T-cat."])
  .apply(highlight_bool, subset=["Mid. ext."])
  .apply(highlight, subset=["ipsi"], mapping={
    "0": COLORS["green"],
    "1": COLORS["orange"],
    "≥ 2": COLORS["red"],
  })
  .hide()
)

# Marginalizing Beta Posteriors over Unknowns

In [7]:
import pandas as pd
import numpy as np
from scripts import shared, COL

data = shared.get_data()
is_early = data[COL.t_stage].isin([0,1,2])

reduced = pd.DataFrame({
  "Midline Extension": data.loc[is_early, COL.midext],
  "contra LNL II": data.loc[is_early, ("max_llh", "contra", "II")]
})

result = (
  reduced
  .fillna("unknown")
  .value_counts(dropna=False)
  .unstack("Midline Extension")
  .rename(
    index={
      False: "healthy",
      True: "involved",
    },
    columns={
      False: "no",
      True: "yes",
    },
  )
)
result.loc["total"] = result.sum()
result

In @sec-prevalence we explained that we compare the predicted prevalence of an involvement pattern under a scenario to a beta posterior over the “true” prevalence given the observations in the data. However, our data contains incomplete observations, especially regarding the tumor’s midline extension. This information is not present for a large group of 85 patients, most of which treated at the Centre Léon Bérard in France.

To account for this missing information and plot the correct posteriors over the data prevalence, we need to marginalize over the possible combinations of matching involvement patterns and selected patients under a scenario. For example, we have only 29<!-- TODO: compute dynamically --> early T-category patients with midline extension. But for an additional 65<!-- TODO: compute dynamically --> patients we do not know whether the tumor crossed the mid-sagittal plane. If we now want to compute the prevalence of contralateral LNL II invovlement for these early T-category patients with midline extension, then we need to account for the possibility that we are missing information.

In @tbl-data-unknowns we show the combinations of the midline extension status and state of the contralateral LNL II. Using only available information, we would get to 6 out of 29<!-- TODO: compute dynamically --> patients for the prevalence of contralateral LNL II invovlement among the early T-category patients with midline extension. But the column with the “unknown” numbers suggests that the true prevalence could range from 6 out of 29 + 59 (7%)<!-- TODO: compute dynamically --> up to 12 out of 29 + 6 (34%)<!-- TODO: compute dynamically -->. We assume that every one of the possible combinations has the same a priori probability (uniform prior) but we do need to account for permutations via the binomial factor. In general, let us say we have $n$ patients with midline extension, of which $k$ harbor metastases in the contralateral LNL II. And we have $m$ patients with unknown midline extension, of which $\ell$ presented with diasease in contralateral LNL II, then the posterior distribution - marginalized over the unknown data - over the true prevalence $x \in \left[ 0, 1 \right]$ is given by:

$$
p(x \mid n, k, m, \ell, \alpha, \beta) = \left[ \frac{1}{m} \sum_{i=0}^m \frac{1}{\min(i,\ell)} \sum_{j=0}^{\min(i,\ell)} \frac{P(i, j \mid x )}{\binom{n}{k}} \right] \cdot P(n, k \mid x) \cdot \frac{1}{\operatorname{B}(\alpha, \beta)} x^\alpha (1-x)^\beta
$$

Note the two binomial likelihoods: one being marginalized over and one acting implicitly as a prior to the summed terms. This role of the likelihood over the data with known midline extension as a prior to the part where that data is missing corrects for the fact that there are highly unlikely combinations among those we marginalize over. E.g., expecting that exactly those 6 patients with contralateral LNL II involvemnt are also coincidentially the only ones of the 65 with a tumor crossing the mid-sagittal line is very unlikely. This is accounted for by the fact that this possibility has only one permutation and it is weakened by the likelihood over the data with known midline extension. With a uniform prior beta distribution ($\alpha=1$ and $\beta=1$), the above simplifies to:

$$
p(x \mid n, k, m, \ell) = \left[ \frac{1}{m} \sum_{i=0}^m \frac{1}{\min(i,\ell)} \sum_{j=0}^{\min(i,\ell)} \binom{n+i}{k+j} x^j (1 - x)^{i-j} \right] \cdot x^{k+1} (1 - x)^{n-k+1}
$$

We have plotted this marginal in @fig-model-marginal-pdf (blue curve) and the terms that make up the above sum (light red curves). Note that we weighted the plotting opacity with the number of permutations each of the possible combinations has. The marginal is compared against the “naive” beta posterior that only takes the data into account for which midline extension is available (green curve).

{ width="50%"}

The distribution that marginalizes over the unknown midline extension cases is shifted towards a lower prevalence. This makes intuitive sense: With only 6 patients among the 65 with unknown midline extension status, the upper bound for the prevalence has moved down compared to 6 out of 29.

We chose to compare our model against such corrected prevalence posteriors, because the model, too, is capable of marginalizing over unknown midline extension in the data. It would therefore be inaccurate to compare the model trained with the full dataset to only that part of the patient cohort where the tumor lateralization was recorded.